# Approach Angle: Flying Experiments

Detect the approach angle while approaching the wall and playing a single frequency.

In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 12

In [ ]:
exp_name = '2021_05_04_linear';
starting_distance = 65.63 # 42+29.7−6.07 

# x, y, z, yaw
starting_poses = {
    '_1': [20, -starting_distance, 0, 45],
    '_2': [-20, -starting_distance, 0, -45],
    '_3': [0, -starting_distance, 0, 0],
    '_4': [-10, -starting_distance, 0, -30],
    '_5': [10, -starting_distance, 0, 30],
    '_fast1': [-20, -starting_distance, 0, -45],
    '_fast2': [20, -starting_distance, 0, 45],
    '_fast3': [0, -starting_distance, 0, 0],
    '_fast4': [20, -starting_distance, 0, 30],
    '_fast5': [-20, -starting_distance, 0, -30]
}
appendices = [f"_{i}" for i in range(1, 6)]#, "_fast2", "_fast4"]
n_rows = 3 
n_max = 14 # number of measurements to combine
step = 13 # plot every step slices 

In [ ]:
exp_name = '2021_10_12_linear';
starting_distance = 100 # 42+29.7−6.07 

# x, y, z, yaw
starting_poses = {
    '': [20, -starting_distance * np.cos(np.pi/4), 0, 45],
    '_1': [20, -starting_distance * np.cos(np.pi/4), 0, 45],
    '_2': [0, -starting_distance, 0, 0],
    '_6': [0, -starting_distance, 0, 0],
    '_8': [-20, -starting_distance * np.cos(np.pi/4), 0, -45],
    '_9': [-20, -starting_distance * np.cos(np.pi/4), 0, -45],
    '_10': [-20, -starting_distance * np.cos(np.pi/4), 0, -45],
}

appendices = ["", "_1", "_2", "_6", "_8", "_9"]#, "_fast2", "_fast4"]

n_rows = 8
n_max = 30 # number of measurements to combine
step = 10 # plot every step slices 

In [ ]:
fname = f'../experiments/{exp_name}/all_data.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except Exception as e:
    print(e)
    answer = input('Run csv_or_wav_parser.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

## 2.1 positions analysis

In [ ]:
def get_average_angle(positions_rot):
    # angles between -180, 180:
    angles = np.arctan2(positions_rot[:, 1]-positions_rot[0, 1], positions_rot[:, 0]-positions_rot[0, 0]) * 180 / np.pi
    
    # convert 120 to 60 etc.
    #angles[angles > 90] = 180 - angles[angles > 90]
    return np.median(angles)

def get_gt_angle(row):
    starting_yaw = starting_poses[row.appendix][3]
    approach_angle = starting_yaw + 90
    return approach_angle

def get_corrected_positions(appendix, positions):
    from scipy.spatial.transform import Rotation as R
    starting_pose = starting_poses[appendix]
    positions_rot = np.empty_like(positions)
    for j, pos in enumerate(positions):
        total_yaw = starting_pose[3] + pos[3]
        rot = R.from_euler('z', total_yaw, degrees=True)
        pos_rot = starting_pose[:3] + rot.apply(pos[:3]) * 1e2
        positions_rot[j, :] = np.r_[pos_rot, total_yaw]
    valid = np.all(~np.isnan(positions_rot), axis=1) & (positions_rot[:, 2] > 35)
    positions_rot = positions_rot[valid, :]
    return positions_rot

def plot_corrected_positions(row, max_idx=30, ax=None, **kwargs):
    if ax is None:
        fig, ax = plt.subplots()
        
    positions_rot = get_corrected_positions(row.appendix, row.positions)
    average_angle = get_average_angle(positions_rot)
    gt_angle = get_gt_angle(row)
    
    ax.plot(positions_rot[:max_idx, 0], positions_rot[:max_idx, 1], 
            label=f'experiment{row.appendix}, {average_angle:.0f} {gt_angle:.0f}', **kwargs)
    ax.axis('equal')
    return positions_rot

In [ ]:
from utils.plotting_tools import plot_df
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

max_idx = 40
fig_df, ax_df = plot_df(distance_range=[7, 100], azimuth_deg=WALL_ANGLE_DEG)

fig, ax = plt.subplots()
fig.set_size_inches(3, 3)
for i, row in df_total.iterrows():
    plot_corrected_positions(row, ax=ax, max_idx=max_idx)
    
    positions_corr = get_corrected_positions(row.appendix, row.positions)
    ds = -positions_corr[:max_idx, 1]
    ax_df.scatter(ds, np.full(len(ds), 3000))
ax.legend(bbox_to_anchor=[1.0, 1.0], loc='upper left')

## 1.2 audio analysis

In [ ]:
from utils.frequency_analysis import add_spectrogram
from utils.plotting_tools import pcolorfast_custom

df_total = df_total.assign(spectrogram=None)
df_total = df_total.apply(add_spectrogram, axis=1)

mic_idx = 0
for i_col, row in df_total.iterrows():
    
    plt.figure()
    for time_idx in range(row.frequencies_matrix.shape[0]):
        mic_idx = 0
        freqs = row.frequencies_matrix[time_idx, :]
        response = np.abs(row.spectrogram[:, mic_idx, time_idx])
        plt.plot(freqs, response)
        
    plt.title('experiment' + row.appendix)

## 1.3 algorithm analysis - old

In [ ]:
from copy import deepcopy
from utils.inference import get_approach_angle_fft
from utils.data_collector import DataCollector
from utils.estimators import AngleEstimator, get_estimate
from utils.plotting_tools import save_fig

plot_raw = False #True #False

mics = [0, 1, 3]
#mics = [1, 3]
n_mics = len(mics)
#n_mics = 4


n_cols = len(appendices)

fig_all, axs_all = plt.subplots(n_rows, n_cols, sharey='row') # 10

[ax.set_xticks([]) for ax in axs_all.flatten()]
[ax.set_yticks([]) for ax in axs_all.flatten()]
axs_all[-1, n_cols//2].set_xlabel('angle [$^\\circ$]')
axs_all[n_rows//2, 0].set_ylabel('probability')
fig_all.set_size_inches(1.5*n_cols, 1.5*n_rows)

fig_res, axs_res = plt.subplots(n_rows, n_cols, sharey='row') # 10
[ax.set_xticks([]) for ax in axs_res.flatten()]
[ax.set_yticks([]) for ax in axs_res.flatten()]
axs_res[-1, n_cols//2].set_xlabel('x [cm]')
axs_res[n_rows//2, 0].set_ylabel('y [cm]')
fig_res.set_size_inches(1.5*n_cols, 1.5*n_rows)
    
#fig_df, ax_df = plot_df(distance_range=[7, 70], freq_range=[2000, 4000])


for k,  appendix in enumerate(appendices):
    row = df_total[df_total.appendix==appendix].iloc[0]
        
    slice_idx = 0
    fig_pos, ax_pos = plt.subplots()
    fig_pos.set_size_inches(3, 3)
    ax_pos.set_xlabel('x [cm]')
    ax_pos.set_ylabel('y [cm]')
    ax_pos.axis('equal')
        
    valid_positions = []
    
    angle_estimator = AngleEstimator()
    data_collector = DataCollector(exp_name=exp_name)
    
    angles = AngleEstimator.ANGLES_DEG
    print(row.stft.shape)
    print(len(angles))
    results_matrix = np.zeros((len(angles), row.stft.shape[0]))
    
    for i in range(row.stft.shape[0]):
        signals_f = row.stft[i]
        frequencies = row.frequencies_matrix[i]
        position_cm = row.positions[i] * 1e2
        
        # need to start new dslice
        if data_collector.next_dslice_ready(signals_f, frequencies, position_cm):
            print('need to start new dslice')
            continue

        # only add measurements if the drone is flying.
        if data_collector.valid_dslice_measurement(
            position_cm, signals_f, frequencies, mono_freq=3000, verbose=False
        ):
            valid_positions.append(position_cm)
            #print('added', position_cm[1])
            data_collector.fill_from_signal(
                signals_f, frequencies, distance_cm=position_cm[1], time=row.seconds[i], 
                #mode="maximum"
                mode=3000
            )

        if (i % step) == 0:
            data =  data_collector.get_current_distance_slice(n_max=n_max)
            if data is not None:
                d_slices, rel_distances, stds, freqs = data
            else:
                #print(f'skipping step, cause no measurements yet')
                continue 
            if len(rel_distances) < n_max:
                #print(f'skipping step, cause only {len(rel_distances)} measurements')
                continue
            print(f'plotting up to {i} of {row.stft.shape[0]}')

            freq = np.mean(freqs)

            positions = np.array(valid_positions[-n_max:]) * 1e-2
            positions_corr = get_corrected_positions(row.appendix, positions)

            mean_angle = get_average_angle(positions_corr)

            ax_pos.scatter(positions_corr[:, 0], 
                           positions_corr[:, 1], 
                           color=f'C{slice_idx}',
                           s=10)
            ax_pos.set_title(f'experiment {k}')

            positions_here = positions_corr - positions_corr[0, :]
            
            axs_res[slice_idx, k].scatter(
                positions_here[:, 0], 
                positions_here[:, 1], 
                color=f'C{slice_idx}',
                s=10, 
                label="trajectory"
            )
            axs_res[0, k].set_title(f'exp. {k}')
            axs_all[0, k].set_title(f'exp. {k}')

            if plot_raw:
                fig, axs = plt.subplots(2, n_mics, sharey='row')
                fig.set_size_inches(10, 7)

            plot_mean_angle = mean_angle if mean_angle < 90 else 180 - mean_angle
            axs_all[slice_idx, k].axvline(plot_mean_angle, color='k', ls=':', label='mean angle')

            for i_mic, mic_idx in enumerate(mics):
                d_slice = d_slices[mic_idx]
                if plot_raw:
                    axs[0, i_mic].set_title(f'mic{mic_idx}')
                    ds = positions_corr[:, 1]
                    axs[0, i_mic].plot(ds, d_slice, color=f'C{slice_idx}', marker='o')
                    axs[0, i_mic].set_xlabel(f"relative distance [cm]")

                valid = ~np.isnan(d_slice)
                angles, proba = get_approach_angle_fft(
                    d_slice=d_slice,
                    frequency=freq,
                    relative_distances_cm=rel_distances,
                    reduced=True,
                    bayes=True
                )
                angle_estimator.add_distribution(angles, proba, mic_idx, freq)

                if plot_raw:
                    axs[1, i_mic].plot(angles, proba, color=f'C{slice_idx}')
                    axs[1, i_mic].axvline(plot_mean_angle, color='k', ls=':', label=mean_angle)
            if plot_raw:
                axs[0, 0].set_ylabel(f"magnitude{row.appendix}")
                axs[1, 0].set_ylabel(f"probability{row.appendix}")

            l=20
            axs_res[slice_idx, k].plot(
                [0, l*np.cos(mean_angle/180*np.pi)],
                [0, l*np.sin(mean_angle/180*np.pi)],
                color='k',
                ls=':',
                label='mean angle'
            )
            
            angles, probs = angle_estimator.get_angle_distribution(
                #mics_left_right=[[1], [3]]
            )
            est_angle = get_estimate(angles, probs)
            
            results_matrix[:, i] = probs

            angles_plot = deepcopy(angles)
            angles_plot[angles_plot > 90] = 180 - angles[angles_plot > 90]
                
            axs_all[slice_idx, k].plot(angles_plot, probs, 
                                    color=f'C{slice_idx}', 
                                    label='estimated', 
                                    ls='-')
            #axs_all[slice_idx].set_yscale('log')

            axs_res[slice_idx, k].plot(
                [0, -l*np.cos(est_angle/180*np.pi)], 
                [0, l*np.sin(est_angle/180*np.pi)],
                color=f'C{slice_idx}',
                ls='-',
                label='estimates'
            )
            axs_res[slice_idx, k].axis('equal')

            slice_idx += 1
            if slice_idx > n_rows - 1: 
                break
                    
    ax_pos.axhline(0, color='k')
    #ax_pos.set_ylim(-100, 2)
    save_fig(fig_pos, f'plots/experiments/new_{exp_name}_pos{row.appendix}.png')
            
save_fig(fig_all, f'plots/experiments/new_{exp_name}_all.png')
save_fig(fig_res, f'plots/experiments/new_{exp_name}_res.png')
save_fig(fig_df,  f'plots/experiments/new_{exp_name}_df.png')

In [ ]:
from copy import deepcopy
from utils.inference import get_approach_angle_fft
from utils.data_collector import DataCollector
from utils.estimators import AngleEstimator, get_estimate

mics = [0, 1, 2, 3]
n_mics = len(mics)
print('using n_max', n_max)

for k,  appendix in enumerate(appendices):
    print("appendix", appendix)
    row = df_total[df_total.appendix==appendix].iloc[0]
        
    data_collector = DataCollector(exp_name=exp_name)
    
    angles = AngleEstimator.ANGLES_DEG
    
    results_matrix = np.zeros((len(angles), row.stft.shape[0]))
    pos_matrix = np.zeros((4, row.stft.shape[0]))
    angle_vector = np.zeros(row.stft.shape[0])
    
    for i in range(row.stft.shape[0]):
        signals_f = row.stft[i]
        frequencies = row.frequencies_matrix[i]
        position_cm = row.positions[i] * 1e2
        
        # need to start new dslice
        #if data_collector.next_dslice_ready(signals_f, frequencies, position_cm):
        #    print('need to start new dslice')
        #    continue

        # only add measurements if the drone is flying.
        if data_collector.valid_dslice_measurement(
            position_cm, signals_f, frequencies, mono_freq=3000, verbose=False
        ):
            pos_matrix[:, i] = position_cm
            #print("fillig", position_cm[1], np.abs(signals_f[0, 15:19]), frequencies[15:19])
            data_collector.fill_from_signal(
                signals_f, frequencies, distance_cm=position_cm[1], time=row.seconds[i], 
                #mode="maximum"
                mode=3000
            )

        data =  data_collector.get_current_distance_slice(n_max=n_max)
        if data is not None:
            d_slices, rel_distances, *_ = data
        else:
            #print(f'skipping step, cause no measurements yet')
            continue 
        if len(rel_distances) < n_max:
            #print(f'skipping step, cause only {len(rel_distances)} measurements')
            continue

        freq = np.mean(freqs)

        positions = deepcopy(pos_matrix[:, i-n_max:i].T) * 1e-2
        positions_corr = get_corrected_positions(row.appendix, positions)
        mean_angle = get_average_angle(positions_corr)

        plot_mean_angle = mean_angle if mean_angle < 90 else 180 - mean_angle
        angle_vector[i] = plot_mean_angle

        angle_estimator = AngleEstimator()
        for i_mic, mic_idx in enumerate(mics):
            d_slice = d_slices[mic_idx]
            valid = ~np.isnan(d_slice)
            angles, proba = get_approach_angle_fft(
                d_slice=d_slice,
                frequency=freq,
                relative_distances_cm=rel_distances,
                reduced=True,
                bayes=True
            )
            angle_estimator.add_distribution(angles, proba, mic_idx, freq)

        angles, probs = angle_estimator.get_angle_distribution(
            #mics_left_right=[[1], [3]]
        )
        est_angle = get_estimate(angles, probs)
        results_matrix[:, i] = probs
        
    distance_idx = np.arange(results_matrix.shape[1])
    gt_angle = starting_poses[row.appendix][3]
    gt_angle = 90 - gt_angle if gt_angle >= 0 else -gt_angle
    
    fig, axs = plt.subplots(1, 2)
    fig.set_size_inches(10, 3)
    axs[0].pcolorfast(distance_idx, angle_estimator.ANGLES_DEG, results_matrix)
    #axs[0].pcolorfast(distance_idx, angle_estimator.ANGLES_DEG, np.log10(results_matrix))
    axs[0].axhline(gt_angle, color='C1')
    axs[0].plot(distance_idx, angle_vector, color='C1', ls=':')
    axs[0].set_title(f'exp {k}')
    
    axs[1].plot(distance_idx, pos_matrix[0], label='x')
    axs[1].plot(distance_idx, pos_matrix[1], label='y')
    axs[1].legend()